# Lab 2 Неперсоналізовані рекомендаційні системи

## import

In [71]:
import pandas as pd
import numpy as np

In [72]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


## Part 1 mean + dump

### code

In [73]:
Ratings_mean=ratings.groupby(['movie_id'])[['rating']].mean().rename(columns = {'rating': 'Mean_rating'})
Ratings_mean

,Mean_rating
movie_id,
1,3.878319
2,3.206107
3,3.033333
4,3.550239
5,3.302326
...,...
1678,1.000000
1679,3.000000
1680,2.000000


In [74]:
Ratings_sum=ratings.groupby(['movie_id'])[['rating']].sum().rename(columns = {'rating': 'sum_rating'})
alpha = 5
Ratings_sum['sum_rating_factor'] = Ratings_sum['sum_rating'] + alpha * (ratings["rating"].mean())
Ratings_sum

,sum_rating,sum_rating_factor
movie_id,,
1,1753,1770.6493
2,420,437.6493
3,273,290.6493
4,742,759.6493
5,284,301.6493
...,...,...
1678,1,18.6493
1679,3,20.6493
1680,2,19.6493


In [75]:
Ratings_count = ratings.groupby(['movie_id'])[['rating']].count().rename(
    columns={'rating': 'count_rating'})
Ratings_count['count_rating_factor'] = Ratings_count['count_rating'] + alpha
Ratings_count

,count_rating,count_rating_factor
movie_id,,
1,452,457
2,131,136
3,90,95
4,209,214
5,86,91
...,...,...
1678,1,6
1679,1,6
1680,1,6


In [76]:
Ratings_damped = pd.merge(Ratings_sum, Ratings_count[[
                          'count_rating', 'count_rating_factor']], on=['movie_id'], how='left')

Ratings_damped['damped_mean'] = Ratings_damped['sum_rating_factor'] / \
    Ratings_damped['count_rating_factor']

Ratings_damped

,sum_rating,sum_rating_factor,count_rating,count_rating_factor,damped_mean
movie_id,,,,,
1,1753,1770.6493,452,457,3.874506
2,420,437.6493,131,136,3.218010
3,273,290.6493,90,95,3.059466
4,742,759.6493,209,214,3.549763
5,284,301.6493,86,91,3.314827
...,...,...,...,...,...
1678,1,18.6493,1,6,3.108217
1679,3,20.6493,1,6,3.441550
1680,2,19.6493,1,6,3.274883


In [77]:
Ratings_mean_dampmean = pd.merge(Ratings_mean, Ratings_damped[[
                                'damped_mean']], on=['movie_id'], how='left')
Ratings_mean_dampmean

,Mean_rating,damped_mean
movie_id,,
1,3.878319,3.874506
2,3.206107,3.218010
3,3.033333,3.059466
4,3.550239,3.549763
5,3.302326,3.314827
...,...,...
1678,1.000000,3.108217
1679,3.000000,3.441550
1680,2.000000,3.274883


### Get TOP N

In [ ]:
topN = 5
result = Ratings_mean_dampmean.nlargest(topN,'damped_mean')
result

,Mean_rating,damped_mean
movie_id,,
318,4.466443,4.450988
408,4.491071,4.449994
483,4.456790,4.438102
64,4.445230,4.429338
169,4.466102,4.428043


## Part 2 - Association

In [91]:
movie_association = pd.DataFrame(
    columns=['movie_id', 'movieId1', 'association'])

distinct_movies = np.unique(ratings['movie_id'])
i = 1
for movie in distinct_movies[1:2]:
    movie_data = ratings[ratings['movie_id'] == movie]

    print("movie:", i, "out of: ", len(distinct_movies))
    j = 1

    for movie1 in distinct_movies:
        movie1_data = ratings[ratings['movie_id'] == movie1]
        # ксть юзерів яким сподобався фільм 1
        distinct_cust = len(np.unique(movie_data['user_id']))
        # ксть юзерів яким сподобався фільм 2
        distinct_cust1 = len(np.unique(movie1_data['user_id']))

        movie2_data = movie1_data[movie1_data['user_id'].isin(
            np.unique([movie_data['user_id']]))]
        
        # ксть юзерів яким сподобався фільм 1 і фільм 2
        distinct_cust_common = len(np.unique(movie2_data['user_id']))

        # fraction_common_cust = float(distinct_cust_common) / float(distinct_cust)
        fraction_common_cust = (float(distinct_cust_common) / float(distinct_cust)) / (
            float(distinct_cust1) / len(np.unique(ratings['user_id'])))

        movie_temp = pd.DataFrame({
            "movie_id": movie,
            "movieId1": movie1,
            "association": fraction_common_cust
        }, index=[0])

        movie_association = pd.concat([movie_association, movie_temp], ignore_index=True)

        if j % 500 == 0:
            print(j)
        j = j + 1

    i = i + 1

movie_association = movie_association.sort_values(
    ['association'], ascending=False)
movie_association

movie: 1 out of:  1682


C:\Users\Kolyanys\AppData\Local\Temp\ipykernel_5904\3492018596.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  movie_association = pd.concat([movie_association, movie_temp], ignore_index=True)


500
1000
1500


,movie_id,movieId1,association
1,2,2,7.198473
1424,2,1425,7.198473
1543,2,1544,7.198473
1434,2,1435,7.198473
1185,2,1186,7.198473
...,...,...,...
1432,2,1433,0.000000
1351,2,1352,0.000000
1348,2,1349,0.000000
1347,2,1348,0.000000
